In [3]:
#对分类表进行分解，将分类信息分开并进行统计
import os
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"] = ["SimHei"]    #用来正常显示中文标签
plt.rcParams["axes.unicode_minus"] = False    #用来正常显示中文标签
if __name__ == '__main__':
    table = r"E:\论文\草地植物分类\植物信息表_model_id.xlsx"
    df = pd.read_excel(table)
    for i in range(df.shape[0]):
        df.loc[i,"界"] = df.loc[i,"分类系统"].split("\n")[0].split(" ")[1].split("(")[0].split("-")[1]
        df.loc[i,"门"] = df.loc[i,"分类系统"].split("\n")[1].split(" ")[1].split("(")[0].split("-")[1]
        df.loc[i,"纲"] = df.loc[i,"分类系统"].split("\n")[2].split(" ")[1].split("(")[0].split("-")[1]
        df.loc[i,"目"] = df.loc[i,"分类系统"].split("\n")[3].split(" ")[1].split("(")[0].split("-")[1]
        df.loc[i,"科"] = df.loc[i,"分类系统"].split("\n")[4].split(" ")[1].split("(")[0].split("-")[1]
        df.loc[i,"属"] = df.loc[i,"分类系统"].split("\n")[5].split(" ")[1].split("(")[0].split("-")[1]
    with pd.ExcelWriter(r"E:\论文\草地植物分类\植物信息表_ClassSystem_ModelID.xlsx") as writer:
        df.to_excel(writer)

In [10]:
#将分类表和推理表进行匹配，方便后续分类进行统计
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"] = ["SimHei"]    #用来正常显示中文标签
plt.rcParams["axes.unicode_minus"] = False    #用来正常显示中文标签
if __name__ == '__main__':
    table = r"E:\论文\草地植物分类\植物信息表_ClassSystem_ModelID.xlsx"
    infer_table = r"E:\论文\草地植物分类\model\MobileNetV3_256_831\train_100epochs\inference_validdata\mobilenetv3_valid.xlsx"
    df = pd.read_excel(table)
    infer_df = pd.read_excel(infer_table)
    infer_merge = pd.merge(infer_df,df,on=["PlantName","PlantName"])
    with pd.ExcelWriter(r"E:\论文\草地植物分类\model\MobileNetV3_256_831\train_100epochs\inference_validdata\mobilenetv3_valid_classsystem.xlsx") as writer:
        infer_merge.to_excel(writer)

In [11]:
##统计每种、属、科的top1和top5分类精度
#将分类表和推理表进行匹配，方便后续分类进行统计
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"] = ["SimHei"]    #用来正常显示中文标签
plt.rcParams["axes.unicode_minus"] = False    #用来正常显示中文标签
if __name__ == '__main__':
    infer_table = r"E:\论文\草地植物分类\model\MobileNetV3_256_831\train_100epochs\inference_validdata\mobilenetv3_valid_classsystem.xlsx"
    infer_df = pd.read_excel(infer_table)
    infer_df_sum = infer_df.groupby(by=["PlantName"])["top1_infer_index","top5_infer_index"].sum()     #统计每个种类top1和top5预测正确数量
    infer_df_sum.rename(columns = {'top1_infer_index' : 'top1_correct_count', 'top5_infer_index' : 'top5_correct_count'}, inplace = True)
    infer_df_count = infer_df.groupby(by=["PlantName"])["top1_infer_index"].count()   #统计每个物种的图像数量
    infer_plant_stastic = pd.concat([infer_df_count,infer_df_sum],axis=1)
    infer_plant_stastic.rename(columns={"top1_infer_index":"valid_iamge_count"})
    infer_plant_stastic["top1_correct_percent"] = infer_plant_stastic["top1_correct_count"] / infer_plant_stastic["top1_infer_index"] * 100    #计算每个物种top1正确率
    infer_plant_stastic["top5_correct_percent"] = infer_plant_stastic["top5_correct_count"] / infer_plant_stastic["top1_infer_index"] * 100    #计算每个物种top5正确率
    with pd.ExcelWriter(r"E:\论文\草地植物分类\model\MobileNetV3_256_831\train_100epochs\inference_validdata\mobilenetv3_valid_ByPlantName_summary.xlsx") as writer:
        infer_plant_stastic.to_excel(writer)

C:\Users\kbqin\AppData\Local\Temp\ipykernel_16876\1983232625.py:10: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  infer_df_sum = infer_df.groupby(by=["PlantName"])["top1_infer_index","top5_infer_index"].sum()     #统计每个种类top1和top5预测正确数量


In [12]:
##统计科的top1和top5分类精度

import pandas as pd
import matplotlib.pyplot as plt

if __name__ == '__main__':
    infer_table = r"E:\论文\草地植物分类\model\MobileNetV3_256_831\train_100epochs\inference_validdata\mobilenetv3_valid_classsystem.xlsx"
    infer_df = pd.read_excel(infer_table)
    family_species_count = infer_df.groupby(by=["科"])["PlantName"].nunique()
    family_species_count.rename("species_count_infamily")
    infer_df_sum = infer_df.groupby(by=["科"])["top1_infer_index","top5_infer_index"].sum()     #统计每个科top1和top5预测正确数量
    infer_df_sum.rename(columns = {'top1_infer_index' : 'top1_correct_count', 'top5_infer_index' : 'top5_correct_count'}, inplace = True)
    infer_df_count = infer_df.groupby(by=["科"])["top1_infer_index"].count()   #统计科物种的图像数量
    infer_df_count.rename("species_image_count")
    infer_plant_stastic = pd.concat([family_species_count,infer_df_count,infer_df_sum],axis=1)
    infer_plant_stastic["top1_correct_percent"] = infer_plant_stastic["top1_correct_count"] / infer_plant_stastic["top1_infer_index"] * 100    #计算每个物种top1正确率
    infer_plant_stastic["top5_correct_percent"] = infer_plant_stastic["top5_correct_count"] / infer_plant_stastic["top1_infer_index"] * 100    #计算每个物种top5正确率
    with pd.ExcelWriter(r"E:\论文\草地植物分类\model\MobileNetV3_256_831\train_100epochs\inference_validdata\mobilenetv3_valid_ByFamily_summary.xlsx") as writer:
        infer_plant_stastic.to_excel(writer)

C:\Users\kbqin\AppData\Local\Temp\ipykernel_16876\3652228417.py:11: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  infer_df_sum = infer_df.groupby(by=["科"])["top1_infer_index","top5_infer_index"].sum()     #统计每个科top1和top5预测正确数量


In [79]:
a = infer_df.groupby(by=["科"])["PlantName"].nunique()

In [86]:
family_species_count

科
五福花科     2
亚麻科      3
伞形科     23
兰科       8
冷蕨科      1
        ..
鸢尾科     12
鸭跖草科     1
麻黄科      5
鼠李科      1
龙胆科     11
Name: PlantName, Length: 88, dtype: int64

In [61]:
#对分类表进行分解，将分类信息分开并进行统计
import os
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"] = ["SimHei"]    #用来正常显示中文标签
plt.rcParams["axes.unicode_minus"] = False    #用来正常显示中文标签
if __name__ == '__main__':
    table = r"E:\论文\草地植物分类\植物信息表_new(831).xlsx"
    df = pd.read_excel(table)
    for i in range(df.shape[0]):
        df.loc[i,"界"] = df.loc[i,"分类系统"].split("\n")[0].split(" ")[1].split("(")[0].split("-")[1]
        df.loc[i,"门"] = df.loc[i,"分类系统"].split("\n")[1].split(" ")[1].split("(")[0].split("-")[1]
        df.loc[i,"纲"] = df.loc[i,"分类系统"].split("\n")[2].split(" ")[1].split("(")[0].split("-")[1]
        df.loc[i,"目"] = df.loc[i,"分类系统"].split("\n")[3].split(" ")[1].split("(")[0].split("-")[1]
        df.loc[i,"科"] = df.loc[i,"分类系统"].split("\n")[4].split(" ")[1].split("(")[0].split("-")[1]
        df.loc[i,"属"] = df.loc[i,"分类系统"].split("\n")[5].split(" ")[1].split("(")[0].split("-")[1]

In [70]:
df.groupby(by=["科"])["PlantName"].count()

科
五福花科     2
亚麻科      3
伞形科     23
兰科       8
冷蕨科      1
        ..
鸢尾科     12
鸭跖草科     1
麻黄科      5
鼠李科      1
龙胆科     11
Name: PlantName, Length: 88, dtype: int64